In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
outbound_data = pd.read_excel("Outbound.xlsx")
demand_data = pd.read_excel("Demand Forecast.xlsx")
unit_data = pd.read_excel('Product Data per State.xlsx')
handling_out_data = pd.read_excel('Warehousing.xlsx')

outbound_data.index = outbound_data.State
demand_data.index = demand_data.state
outbound_data = outbound_data.drop(['State','Small shipment'], axis=1)

as_is_dc = {
    "NY": ['CT', 'DC', 'DE', 'MA', 'MD', 'ME', 'NH', 'NJ', 'NY', 'PA', 'RI', 'VT'],
    "ND": ['MN', 'MT', 'ND', 'SD', 'WY'],
    "IL": ['HI', 'IA', 'IL', 'IN', 'KS', 'KY', 'MI', 'MO', 'NE', 'OH', 'VA', 'WI', 'WV'],
    "TN": ['AL', 'FL', 'GA', 'NC', 'SC', 'TN'],
    "WA": ['AK', 'ID', 'OR', 'WA'],
    "TX": ['AR', 'AZ', 'LA', 'MS', 'NM', 'OK', 'TX'],
    "UT": ['CA', 'CO', 'NV', 'UT']
}

dc_product_demand = {}
for dc in as_is_dc:
    dc_product_demand[dc] = {}  # Create a dictionary for each DC
    for product in ['blender', 'swing', 'chair', 'scooter', 'skiprope']:
        total_demand_per_dc = 0
        for state in as_is_dc[dc]:
            total_demand_per_dc += (demand_data.loc[state, product])
        dc_product_demand[dc][product] = total_demand_per_dc  # Store demand (in boxes) for each product based on DC

print(dc_product_demand)

{'NY': {'blender': 226319, 'swing': 6468, 'chair': 161655, 'scooter': 3233, 'skiprope': 969935}, 'ND': {'blender': 31949, 'swing': 913, 'chair': 22820, 'scooter': 22832, 'skiprope': 136921}, 'IL': {'blender': 271812, 'swing': 7766, 'chair': 194150, 'scooter': 3884, 'skiprope': 1164904}, 'TN': {'blender': 216583, 'swing': 6188, 'chair': 154702, 'scooter': 3095, 'skiprope': 928213}, 'WA': {'blender': 51437, 'swing': 1470, 'chair': 36741, 'scooter': 735, 'skiprope': 220442}, 'TX': {'blender': 190768, 'swing': 5451, 'chair': 136263, 'scooter': 2725, 'skiprope': 817575}, 'UT': {'blender': 179502, 'swing': 5129, 'chair': 128215, 'scooter': 2564, 'skiprope': 769295}}


In [4]:
warehousing_handlingOut_data = pd.read_excel('Warehousing.xlsx', sheet_name='Handling Out')
warehousing_handlingIN_data = pd.read_excel('Warehousing.xlsx', sheet_name='Handling In', skiprows=1)
#warehousing_storageCost = pd.read_excel('Warehousing.xlsx', sheet_name='Storage', skiprows= 1) 

divided_dict = {}
for key, nested_dict in dc_product_demand.items():
    divided_dict[key] = {nested_key: np.ceil(nested_value / 365) for nested_key, nested_value in nested_dict.items()}

print(divided_dict)

products = ['blender', 'swing', 'chair', 'scooter', 'skiprope']
volume_perBox = [0.0070, 0.0045, 0.0098, 0.0200, 0.0055]
average_stock_level_per_product = [ 20, 30, 25, 10, 40]

volumes_perStates = {}

averagestockLevelForProductPerState = {}

for state, products_dict in divided_dict.items():
    averagestockLevelForProductPerState[state] = {}
    for i, product in enumerate(products):
        num_boxes = products_dict[product]
        hashtagdays = average_stock_level_per_product[i]
        averagestockLevelForProductPerState[state][product] = np.ceil(num_boxes * hashtagdays)

print(averagestockLevelForProductPerState)

for state, products_dict in averagestockLevelForProductPerState.items():
    volumes_perStates[state] = {}
    for i, product in enumerate(products):
        num_boxes = products_dict[product]
        volume = volume_perBox[i]
        volumes_perStates[state][product] = np.ceil(num_boxes * volume)


print(volumes_perStates)

{'NY': {'blender': 621.0, 'swing': 18.0, 'chair': 443.0, 'scooter': 9.0, 'skiprope': 2658.0}, 'ND': {'blender': 88.0, 'swing': 3.0, 'chair': 63.0, 'scooter': 63.0, 'skiprope': 376.0}, 'IL': {'blender': 745.0, 'swing': 22.0, 'chair': 532.0, 'scooter': 11.0, 'skiprope': 3192.0}, 'TN': {'blender': 594.0, 'swing': 17.0, 'chair': 424.0, 'scooter': 9.0, 'skiprope': 2544.0}, 'WA': {'blender': 141.0, 'swing': 5.0, 'chair': 101.0, 'scooter': 3.0, 'skiprope': 604.0}, 'TX': {'blender': 523.0, 'swing': 15.0, 'chair': 374.0, 'scooter': 8.0, 'skiprope': 2240.0}, 'UT': {'blender': 492.0, 'swing': 15.0, 'chair': 352.0, 'scooter': 8.0, 'skiprope': 2108.0}}
{'NY': {'blender': 12420.0, 'swing': 540.0, 'chair': 11075.0, 'scooter': 90.0, 'skiprope': 106320.0}, 'ND': {'blender': 1760.0, 'swing': 90.0, 'chair': 1575.0, 'scooter': 630.0, 'skiprope': 15040.0}, 'IL': {'blender': 14900.0, 'swing': 660.0, 'chair': 13300.0, 'scooter': 110.0, 'skiprope': 127680.0}, 'TN': {'blender': 11880.0, 'swing': 510.0, 'chair'

In [5]:

# Cost per cubic meter per month for each state
cost_per_m3 = {
    'NY': 60,
    'ND': 10,
    'IL': 27,
    'TX': 19,
    'MO': 15,
    'CA': 40,
    'TN': 10,
    'PA': 30,
    'UT': 20,
    'WA': 29
}

# Initialize a dictionary to store the total sum for each state
total_sums = {}
# Initialize a dictionary to store the total cost for each state
total_costs = {}

# Calculate the total volume for each state
for state, items in volumes_perStates.items():
    total_sum = sum(items.values())
    total_sums[state] = total_sum



# Calculate the total cost for each state
for state, total_sum in total_sums.items():
    if state in cost_per_m3:
        total_costs[state] = total_sum * cost_per_m3[state]

total_cost_sum = sum(total_costs.values())
print(total_cost_sum*12)

#print(warehousing_storageCost['DC'])
print(cost_per_m3)

1341612.0
{'NY': 60, 'ND': 10, 'IL': 27, 'TX': 19, 'MO': 15, 'CA': 40, 'TN': 10, 'PA': 30, 'UT': 20, 'WA': 29}
